# Recruitment Tree Diagram
<b>Author: A.Krauskopf</b><br>
<b>Last Update: 20 Oct 2025</b>

This notebook includes the code generation for a sfdp tree based on a sql database and output a .svg vector file. It subsequently post-processes the vector file to replace the founders and super-recruiter nodes with specific .svg files.

<b>Revision Notes:</b>
- Correct bug which was including OptOut players
- Update label logic for RIP modification
- Colors now driven by 2025 roster or pronouns
- Hidden logic for gravity nodes to connect founders for final layout
- Update to output (3) grouped subgraphs

<b>NOTE: This code is included in the repo for reference only as the full database has not been publicly released to maintain individual privacy.</b>

In [7]:
# Import the necessary Packages
import sqlite3
from graphviz import Digraph
from collections import defaultdict
import xml.etree.ElementTree as ET
import copy
import os

In [8]:
# Create a sfdp tree from the sql database data

# Define the founders / root players
founders = [1, 2, 3]
print(founders)

# Connect to the database
conn = sqlite3.connect('sauceball.db')
cursor = conn.cursor()

# Get player data (omitting OptOut = 'Y')
cursor.execute("""
  SELECT PlayerID, FirstName, Nickname, Surname, Pronouns, RookieYear, 
         RecruitedByID, RIP
  FROM Player
  WHERE (RecruitedByID IS NOT NULL OR PlayerID IN (?, ?, ?))
    AND (OptOut IS NULL OR OptOut = 'N')
""", founders)
rows = cursor.fetchall()

# Identify all players in the current year's roster
cursor.execute("""
    SELECT DISTINCT R.PlayerID
    FROM Roster R
    JOIN Team T ON R.TeamID = T.TeamID
    WHERE T.Year = 2025
""")
current_players = set(row[0] for row in cursor.fetchall())

# Determine each players' final active year for RIP logic
cursor.execute("""
    SELECT PlayerID, MIN(T.Year), MAX(T.Year)
    FROM Roster R
    JOIN Team T ON R.TeamID = T.TeamID
    GROUP BY PlayerID
""")
player_years = {pid: (min_yr, max_yr) for pid, min_yr, max_yr in cursor.fetchall()}

conn.close()

# Build player dictionary for node labels
players = {}
for pid, fn, nn, sn, pronouns, ry, recruited, rip in rows:
    full_name_length = len(f'{fn} {sn}')

    # Special handling for RIP players
    if rip and rip.upper() == 'Y':
        if pid in player_years:
            first_year, last_year = player_years[pid]
            year_label = f"{first_year}-{last_year}"
        else:
            # Fallback in case no roster data found
            year_label = f"{ry or 'N/A'}–?"
    else:
        # Normal label for living players
        if ry == 1990:
            year_label = "1990 OG"
        else:
            year_label = f'{ry or "N/A"} Rookie'

    # Build the name label
    if nn:
        label = f'#{pid} {fn}\n"{nn}"\n{sn}\n{year_label}'
    else:
        if full_name_length > 15:
            label = f'#{pid} {fn}\n{sn}\n{year_label}'
        else:
            label = f'#{pid}\n{fn} {sn}\n{year_label}'

    players[pid] = {
        'label': label,
        'pronouns': pronouns,
        'rookie_year': ry,
        'recruited_by': recruited,
        'rip': rip
    }

# Build recruitment relationships
tree = defaultdict(list)
roots = []
for pid, data in players.items():
    recruiter = data['recruited_by']
    if recruiter and recruiter in players:
        tree[recruiter].append(pid)
    else:
        roots.append(pid)

# Detect super-recruiters (≥4 recruits)
superRecruiters = {pid for pid, recruits in tree.items() if len(recruits) >= 4}
print(superRecruiters)

# Initialize Graphviz using the sfdp engine
dot = Digraph('FamilyTree', engine='sfdp', graph_attr={
    'overlap': 'false',
    'splines': 'true',
    'smoothing': 'true',
    #'dpi': '150', # default is 96
    'nodesep': '0.5',
    'ranksep': '0.5',
})

# Set default attributes
dot.attr('node', fontsize='11', fontname='Silk Flower', style='filled')
dot.attr('edge', color='#6E695C', penwidth='2', arrowhead='dot')

# Add nodes in subgraphs per founder
def add_subtree(c, pid):
    """Recursively add a node and its subtree to a subgraph."""
    d = players[pid]

    # Color logic
    if pid in current_players:
        fillcolor = '#AF4A3B'
    else:
        fillcolor = {
            'He/Him': '#6FA059',
            'She/Her': '#A5CC46',
            'They/Them': '#C7DD71'
        }.get(d['pronouns'], '#AAAAAA')  # default gray

    node_kwargs = {
        'label': d['label'],
        'fillcolor': fillcolor,
        'fontcolor': 'black',
        'style': 'filled',
        'shape': 'ellipse',
        'fontsize': '7',
        'penwidth': '1',
        'fixedsize': 'false',
        'margin': '0'
    }

    if pid in founders:
        node_kwargs.update({
            'width': '1.5',
            'height': '1.5',
            'shape': 'circle',
            'fixedsize': 'true',
            'fontsize': '10'
        })
    elif pid in superRecruiters:
        node_kwargs.update({
            'width': '1',
            'height': '1',
            'shape': 'circle',
            'fixedsize': 'true',
            'fontsize': '8'
        })

    c.node(str(pid), **node_kwargs)

    # Recursively add children
    for child in tree.get(pid, []):
        add_subtree(c, child)

# Add each root/founder in a separate invisible cluster
for root_pid in roots:
    with dot.subgraph(name=f'cluster_{root_pid}') as c:
        c.attr(style='invis', label='')
        add_subtree(c, root_pid)

# Add recruitment edges
for recruiter, recruits in tree.items():
    for recruit in recruits:
        dot.edge(str(recruiter), str(recruit))
        
# Render initial graphics with png preview
folder = 'outputs'
filename = 'familytree'

png_path = os.path.join(folder, filename)
dot.format = 'png'
dot.render(png_path, view=True)

svg_path = os.path.join(folder, filename)
dot.format = 'svg'
dot.render(svg_path, view=False)

[1, 2, 3]
{1, 2, 3, 4, 641, 6, 7, 8, 136, 390, 514, 1033, 1162, 1157, 1168, 657, 1148, 1296, 148, 404, 406, 660, 1172, 27, 29, 30, 1057, 162, 34, 674, 806, 1065, 1067, 941, 817, 307, 1334, 824, 59, 317, 320, 586, 714, 1100, 718, 79, 336, 1108, 599, 1111, 857, 859, 220, 349, 734, 987, 739, 228, 234, 1003, 1258, 366, 879, 1006, 241, 1268, 503, 764, 510}


'outputs/familytree.svg'

In [12]:
# Define the methods for post-processing

# Load and parse a .svg file to determine its viewBox parameters (minX minY width height)
def loadSvg(svg_file):
    tree = ET.parse(svg_file)
    root = tree.getroot()

    viewbox = root.get('viewBox')
    if viewbox:
        parts = viewbox.split()
        width = float(parts[2])
        height = float(parts[3])
    else:
        width = root.get('width', '100')
        height = root.get('height', '100')
        width = float(''.join(c for c in str(width) if c.isdigit() or c == '.'))
        height = float(''.join(c for c in str(height) if c.isdigit() or c == '.'))

    return root, width, height

# Replace founders and super-recruiter nodes with pre-colored SVGs.
def replaceNodes(input_svg, output_svg, founders, super_recruiters, current_players):
    print("🔍 Starting replaceNodes()...")

    ET.register_namespace('', 'http://www.w3.org/2000/svg')
    ET.register_namespace('xlink', 'http://www.w3.org/1999/xlink')

    tree = ET.parse(input_svg)
    root = tree.getroot()
    print(f"🔍 Parsed input SVG: {input_svg}")

    # Load all SVG files for founders and super-recruiters
    svg_files = [
        "flowerCurrent.svg", "flowerHe.svg", "flowerShe.svg", "flowerThey.svg",
        "appleCurrent.svg", "appleHe.svg", "appleShe.svg", "appleThey.svg"
    ]
    svg_folder = "inputs"

    svgs = {}
    for filename in svg_files:
        filepath = os.path.join(svg_folder, filename)
        svg_root, width, height = loadSvg(filepath)
        name = filename.replace('.svg', '')
        svgs[name] = {'root': svg_root, 'width': width, 'height': height}

    # Map node colors to pronoun variant for identification
    color_map = {
        '#6fa059': 'He',     # He/Him
        '#a5cc46': 'She',    # She/Her
        '#c7dd71': 'They',   # They/Them
        '#af4a3b': 'Current' # 2025 special color
    }

    ns = {'svg': 'http://www.w3.org/2000/svg'}
    nodes = root.findall('.//svg:g[@class="node"]', ns)
    print(f"🔍 Found {len(nodes)} node groups in SVG.")

    modified_count = 0

    for g in nodes:
        title = g.find('svg:title', ns)
        if title is None:
            continue

        try:
            node_id = int(title.text)
        except (ValueError, AttributeError):
            continue

        ellipse = g.find('svg:ellipse', ns)
        if ellipse is None:
            continue

        cx = float(ellipse.get('cx'))
        cy = float(ellipse.get('cy'))
        rx = float(ellipse.get('rx'))
        ry = float(ellipse.get('ry'))

        # Determine base type
        if node_id in founders:
            base = "flower"
        elif node_id in super_recruiters:
            base = "apple"
        else:
            continue  # skip nodes that are neither

        # Read fill color and map to variant
        fill_color = ellipse.get('fill', '#AAAAAA').lower()
        variant = color_map.get(fill_color, 'Current')

        # Override for season_2025_players
        if node_id in current_players:
            variant = 'Current'

        key = f"{base}{variant}"
        svg_info = svgs.get(key)
        if not svg_info:
            print(f"Missing SVG for key: {key}")
            continue

        custom_svg_root = svg_info['root']
        svg_width = svg_info['width']
        svg_height = svg_info['height']

        # Compute target size and offset
        target_size = rx * 2
        y_offset_percent = 0
        if base == "apple":
            target_size *= 1.42
            y_offset_percent = 15 / (rx * 2) # offset center for apple shape

        # Replace ellipses with custom SVG for founders and super-recruiters
        g.remove(ellipse)
        svg_group = ET.Element('{http://www.w3.org/2000/svg}g')

        scale_x = target_size / svg_width
        scale_y = target_size / svg_height
        scale = min(scale_x, scale_y)

        scaled_width = svg_width * scale
        scaled_height = svg_height * scale
        translate_x = cx - scaled_width / 2
        translate_y = cy - scaled_height / 2 - (scaled_height * y_offset_percent)
        svg_group.set('transform', f"translate({translate_x}, {translate_y}) scale({scale})")

        for child in custom_svg_root:
            svg_group.append(copy.deepcopy(child))

        g.insert(0, svg_group)
        modified_count += 1
        print(f"Replaced ellipse for node {node_id} using {key}")

    print(f"\Done! Modified {modified_count} nodes total.")
    tree.write(output_svg, encoding='utf-8', xml_declaration=True)
    print(f"Saved modified SVG to {output_svg}")

In [13]:
# Call the defined methods for post-processing of the svg file

replaceNodes(
    input_svg='outputs/familytree.svg',
    output_svg='outputs/familytree_custom.svg',
    founders=founders,
    super_recruiters=superRecruiters,
    current_players=current_players,
)

🔍 Starting replaceNodes()...
🔍 Parsed input SVG: outputs/familytree.svg
🔍 Found 1174 node groups in SVG.
Replaced ellipse for node 1 using flowerHe
Replaced ellipse for node 4 using appleHe
Replaced ellipse for node 7 using appleHe
Replaced ellipse for node 8 using appleHe
Replaced ellipse for node 79 using appleHe
Replaced ellipse for node 234 using appleHe
Replaced ellipse for node 366 using appleHe
Replaced ellipse for node 404 using appleCurrent
Replaced ellipse for node 1334 using appleCurrent
Replaced ellipse for node 136 using appleShe
Replaced ellipse for node 220 using appleHe
Replaced ellipse for node 148 using appleHe
Replaced ellipse for node 241 using appleCurrent
Replaced ellipse for node 739 using appleShe
Replaced ellipse for node 879 using appleCurrent
Replaced ellipse for node 817 using appleCurrent
Replaced ellipse for node 349 using appleHe
Replaced ellipse for node 599 using appleShe
Replaced ellipse for node 307 using appleHe
Replaced ellipse for node 510 using ap